# Notebook 2

## Project 5

#### Quang, Paul and Luke

With our token acquired from the youtube api, we set about acquiring data that interested us from youtube. To that end, we decided to primarily focus on time of upload, and categories of video. For each video uploaded, youtube and the content creator assigns a category id to the video, denoting the type of video you will be watching. These broad categories range from politics, to entertainment, animals and pets, and everything in between. We decided to get our data from the mostPopular videos of each day, so we could see if there was a prime upload time, either in general or specific to each category, that would lend a video a more likely chance of success. 

Our data acquisition process comes from one primary main() function, with 12 subsidary helper functions. This notebook will take you through these functions, and in turn, how we went about getting and organizing our data.

In [ ]:
import json
import requests
import pandas as pd
import importlib
from requests_oauthlib import OAuth2Session
import keys
from sqlalchemy import *
import keysSQL

This is our main function. Initially, when we were going about our exploration of the youtube api, we were only looking at the most popular videos globally. However, as the project progressed, we decided that it would be interesting to not only find data for the most popular videos specific to a region, but look at and compare the most popular videos between a few regions at once, without having to create multiple relational tables. Therefore, we designed our main() function so it could either take no inputs and default to gather global data, or take any number of specified regions, and gather all of their data.

In [ ]:
def main(countries = []):
    """
    This function gathers data from the youtube api about their current
    most popular videos, and can be narrowed to look at either a specific
    region, or a group of them. It then sends the data to an sql database
    
    Parameters:
    Optionally a list of countries to gather local data
    
    Return Values:
    None
    """
    if countries == []:
        df=query()
    #gathers global data if no country is specified
        
        engine,connection,metadata=sqlLoadIn()
        toSql(engine,connection,metadata,df)
    
    else:
        country_concat = []
        string=''
        for country in countries:
            string+=country+'_'
            df = query(country)
            country_concat.append(df)
    #gathers each countries data from the parameter lists, and compiles
    #them in one master dictionary
    
        df=pd.concat(country_concat, ignore_index=True)
    
        engine,connection,metadata=sqlLoadIn()
        toSql(engine,connection,metadata,df,string)

After our main() function, the first helper function hierarchically is our query() function. This function serves a few purposes. First, it collects our authorization token from our loadIn() function, and then tests if our token is still valid for collecting data. If not, it automatically gathers a refreshed token, and goes through the acquisition process again. Finally, it merges the data pulled from the youtube api with another table we've generated to match the category idea with the name of what each id means. 

In [ ]:
def query(country=""):
    """
    This function queries the youtube api for data about their current
    most popular videos, and returns that information in a dataframe
    
    Parameters:
    Optionally a country to gather local data
    
    Return Values:
    A dataframe
    """
    token=loadIn()
    try:
        df=getData(token,country)
    #this try except loops tests if the authorization token is valid,
    #and refreshes it if not
        
        result=merge(token,df,country)
        return result
    
    except KeyError:
        new_token=refreshToken(token)
    
        df=getData(new_token,country)
        result = merge(new_token,df,country)
        return result

refreshToken() does exactly what the name says, and refreshes our token if it is currently expired. To do this we need access to our keys.py, as well as our current OAuth session and refresh_url, all of which we get from the helper function keychain().

In [ ]:
def refreshToken(token):
    """
    This function takes an expired token, and refreshes the expired token
    with the refresh token
    
    Paremeters:
    An expired token
    
    Return Value:
    A new token
    """
    keychain_1,session,refresh_url=keychain()
    #Uses the helper function keychain to pull needed information out to refresh
    #the token
    
    token = session.refresh_token(refresh_url, 
                                 client_id=keychain_1['youtube']['client_id'],
                                 client_secret=keychain_1['youtube']['client_secret'],
                                 refresh_token=token['refresh_token'])
    #creates a refreshed session with the youtube api using the refresh token
    #to allowed continued access and authorization
    return token

keychain() shares the burden with loadIn() for setting up most of the structural framework we'll need for this project. The main point of this function is setting up an OAuth2 session, but it also takes care of a few specifics which can then be passed to refreshToken() to let it run properly.

In [ ]:
def keychain():
    """
    This function opens a keys.py file in the same directory, and using that keychain, creates an OAuth2 session with our api
    
    Parameters:
    None
    
    Return Values:
    The keys.py file as keychain, an OAuth2 session, and a refresh_url
    """
    importlib.reload(keys)
    keychain = keys.keychain
    client_id = keychain['youtube']['client_id']
    scope = keychain['youtube']['scope']
    redirect = keychain['youtube']['redirect_uris'][0]
    refresh_url = keychain['youtube']['token_uri']
    session = OAuth2Session(client_id, scope=scope, redirect_uri=redirect)
    return keychain,session,refresh_url

Here is the function we used to load in our token from our previous notebook. Using this method, we can access it at any point throughout this notebook, or any others we would need to.

In [ ]:
def loadIn():
    """
    This function opens a saved token.json file in the same directory, and reads it in in json
    
    Parameters:
    None
    
    Return Values:
    A json object
    """
    token=open('token.json',mode='r')
    token=token.read()
    token=json.loads(token)
    #turns the token from a string object into a json one
    return token

As the name denotes, getData() is our primary data gathering function. Once this function takes a valid token, passed to it from query(), it uses a helper function to generate the proper url to use a get request to pull the data from the youtube api, and then uses another helper function to arrange it into a tidy dataframe. This tidy dataframe with the youtube data is then passed back to query().

In [ ]:
def getData(token,country=""):
    """
    This function, with the use of several helper functions, 
    queries the youtube api, retrieves data, and sorts it into a dictionary.
    This function is the parent function of createUrl and arrange.
    
    Parameters:
    A valid authorization token, and optionally a country to gather local data 
    
    Return Values:
    A dataframe
    """
    data = {'videoId':[], 'channelId':[],'categoryId':[] ,'channelTitle':[], 'publishedAt':[]}
    page = []
    popular_vid,D,url=createUrl(token,country)
    #Uses the helper function createUrl to get information from the youtube api
    
    for i in range(10):
        if 'nextPageToken' not in popular_vid:
            next_page = 'CAUQAA'
        else:
            next_page = popular_vid['nextPageToken']
    #creates a for loop to loop through consecutive pages of information from the youtube api
    #using the nextPageToken
        
        page.append(next_page)
        D['pageToken'] = next_page
        popular_vid = requests.get(url, params = D)
    #requests new pages of information from the youtube api by appending the url
        
        popular_vid = popular_vid.json()
        arrange(data,popular_vid['items'],country)
    return pd.DataFrame(data)

Our createUrl() function, as the names implies, creates the url we will be using to make a get request to the youtube api. Additionally however, it also passes a dictionary back to it's parent function with the parameters of the url so the information gathered can be more easily sorted, as well as a json object with the initial page of information from the api.

In [ ]:
def createUrl(token,country=""):
    """
    This function creates a url, which it uses as part of a get 
    request to query the youtube api for information about the most
    popular videos in a given region.
    
    Parameters:
    A valid authorization token, and optionally a country to gather local data 
    
    Return Values:
    A json object with the queried data, a dictionary with the search paramters, and the created url
    """
    D={}
    D['access_token'] = token['access_token']
    D['part'] = ['snippet,contentDetails,statistics']
    D['maxResults'] = '50'
    D['chart'] ='mostPopular'
    D['PageToken'] = 'CAUQAA'
    if country != "":
        D["regionCode"] = country
    #checks if a country was specified
    
    url ='https://www.googleapis.com/youtube/v3/videos'
    popular_vid = requests.get(url, params = D)
    #requests the information for the youtube api
    
    popular_vid = popular_vid.json()
    #turns the recieved information into a json object
   
    return popular_vid,D,url

Lastely from getData() is our arrange() function. The functionality of arrange() is farily simple. It takes all of the data we have gathered thus far from the api as raw json, and sorts it into a dictionary before returning it back to getData(), and subsequently, query(). Having it in this form allows us to easily put it in a dataframe, and then upload it to a sql datbase.

In [ ]:
def arrange(data,vid,country=""):
    """
    This function sorts the gathered data into a dictionary, so it can later be passed into a data frame
    
    Parameters:
    A dictionray, the gathered data, and optionally the country the data was gathered from
    
    Return Values:
    A dictionary
    """
    if country != "" and "CountryCode" not in data:
        data["CountryCode"] = []
    #checks if a country was specified
    
    for item in vid:
        data['videoId'].append(item['id'])
        data['channelId'].append(item['snippet']['channelId'])
        data['categoryId'].append(item['snippet']['categoryId'])
        data['channelTitle'].append(item['snippet']['channelTitle'])
        data['publishedAt'].append(item['snippet']['publishedAt'][11:13])
        if country != "":
            data['CountryCode'].append(country)
    return data

merge() is a parent function to two seperate functions, categories() and catgoryTable(), who's purpose is to take our earlier ditctionary with all our data on the popular regional videos, and merge it with a new dictionary pairing category ids to their string, resulting in one unified dataframe. 

In [ ]:
def merge(token,df,country=""):
    """
    This function, with the use of several helper functions,
    queries the youtube api so it can merge two dataframes
    together, and apply correct category labels.
    
    Paramters:
    A valid authorization token, a dataframe, and optionally 
    a country
    
    Return Values:
    A dataframe
    """
    category_vid=categories(token)
    df1=categoryTable(category_vid['items'])
    
    result = pd.merge(df, df1, left_on='categoryId', right_on='id')
    return result 

categories() serves a similar function as getData(), but less complicated. This function sends a get request to the youtube api, but instead of gathering live data on videos, it pulls information about the category ids on youtube, passes it into a json ojbect, and then returns it.

In [ ]:
def categories(token,country=""):
    """
    This function queries the youtube api for the names of each categoryID
    
    Paramters:
    A valid authorization token, and optionally a country to gather local data 
    
    Return Value:
    A json object
    """
    url = 'https://www.googleapis.com/youtube/v3/videoCategories'
    C = {}
    C['access_token'] = token['access_token']
    C['part'] = 'snippet'
    if country!="":
        C['regionCode']=country
    #checks if a country was specified
    
    C['regionCode'] = 'US'
    category_vid = requests.get(url, params = C)
    category_vid = category_vid.json()
    return category_vid

categoryTable() takes a json object, in this case the one created by categories(), and converts it from json to a dictionary.

In [ ]:
def categoryTable(json):
    """
    This function takes a json object and sorts it into a dictionary
    
    Parameters:
    A json object
    
    Return Values:
    A dataframe
    """
    data_category = {'id':[],'title':[]}
    for item in json:
        data_category['id'].append(item['id'])
        data_category['title'].append(item['snippet']['title'])
    return pd.DataFrame(data_category)

At this point, we have gotten the data we wanted from the youtube api, sorted it into a dictionary, and merged it together into one unified data frame, so each row gives us the infomration about the video, the time uploaded, and what category of video the upload was. Initally when we reached this point, we exported the data as a .csv file, and uploaded that information locally to tableau to begin visualizing what we'd learned and gathered from our data acquisition. However, we decided that we might be better served by instead creating a a connection to an sql server, and uploading a database of our data to the server, and accessing it through there. 

sqlLoadIn() creates the necessary engine, connection, and metadata to connect to a sql server, as well as upload information. It then returns these pieces so a subsequent function can access them for pushing things to the this server.

In [ ]:
def sqlLoadIn():
    """
    This function loads in the necessary data to connect to an
    sql server
    
    Parameters:
    None
    
    Return Values:
    An engine object, a connection, metadata
    """
    keychain = keysSQL.keychain
    protocol = "mysql+mysqlconnector"
    userid = keychain["MySQL"]["userid"]
    userpass = keychain["MySQL"]["userpass"]
    mysqlhost = "hadoop2.mathsci.denison.edu"
    database = "rubens_p1"
    connectionstring = "{}://{}:{}@{}/{}".format(protocol, userid, userpass, mysqlhost, database)
    engine = create_engine(connectionstring)
    connection = engine.connect()
    metadata = MetaData()
    return engine,connection,metadata

Lastly, toSql() sends the dataframe we created earlier to the sql server we have established the connection to. The main issue that needed solving here, and what made the function so long, was accounting for whether countries had been specified in the initial paramter of main(). If it had, we first needed to change the table we were creating to include, or exclude a CountryCode column, to keep the lenght of the table the same as the dataframe. Then, we had to make sure that each table had a unique name when it was uploaded to the server, as to not overwite previously uploaded, but different talbes.

In [ ]:
def toSql(engine,connection,metadata,df,loc='global'):
    """
    This function takes a connection to an sql server and a
    dataframe, and turns the dataframe object into a database
    on the server.
    
    Parameters:
    An engine to database, a connection to a database,
    metadata for managing the connection, a dataframe, and 
    optionally a country name if specified.
    
    Return Values:
    None
    """
    if loc!='global':
    #checks if countires have been specified, and gives the database
    #a unique name if so
    
        dropTable = text("DROP TABLE IF EXISTS "+loc)
        connection.execute(dropTable)
    #This sees if an identically named table exists, and deletes it if
    #that's the case to avoid errors

        country_cate = Table(loc, metadata,
                         Column('CountryCode', String(255)),
                         Column('categoryId', Integer()),
                         Column('channelId', String(255)),
                         Column('channelTitle', String(255)),
                         Column('publishedAt', Integer()),
                         Column('videoId', String(255)),
                         Column('Id', Integer()),
                         Column('title', String(255)),extend_existing=True)

        metadata.create_all(engine)
        columnList = df.values.tolist()

        for row in range(len(df)):
            stmt = insert(country_cate).values(CountryCode = columnList[row][0], 
                                       categoryId = columnList[row][1], 
                                       channelId = columnList[row][2],
                                       channelTitle = columnList[row][3], 
                                       publishedAt = columnList[row][4],
                                       videoId = columnList[row][5],
                                       Id = columnList[row][6],
                                       title = columnList[row][7])

            connection.execute(stmt)
    
    else:
    #if no country is specified, the CountryCode column needs to be removed to
    #keep the index in range
    
        dropTable = text("DROP TABLE IF EXISTS global")
        connection.execute(dropTable)
        
        country_cate = Table("global", metadata,
                        Column("categoryId", Integer()),
                        Column("channelId", String(255)),
                        Column("channelTitle", String(255)),
                        Column("publishedAt", Integer()),
                        Column("videoId", String(255)),
                        Column("Id", Integer()),
                        Column("title", String(255)),extend_existing=True)
        
        metadata.create_all(engine)
        columnList = df.values.tolist()

        for row in range(len(df)):
            stmt = insert(country_cate).values(categoryId = columnList[row][0], 
                                       channelId = columnList[row][1],
                                       channelTitle = columnList[row][2], 
                                       publishedAt = columnList[row][3],
                                       videoId = columnList[row][4],
                                       Id = columnList[row][5],
                                       title = columnList[row][6])
            
            connection.execute(stmt)
   
    connection.close()

In [ ]:
main(['ES','FR'])

Ultimately, this leaves us with one main() function, and a database table on an sql server. As our scope of what we wanted to include in the project evolved, so to did the complexity. Changing the notebook from something unorganized and full of gloabl varialbes into one with one main function with subsidary helper functions provided some unforseen challenges, as we had to make sure that each helper function had the appropriate token, and could handle if whether or not we had specified countries. One of the last changes we did was implementing the ability to pull information on multiple countries into one dataframe, which required us to turn query, our old main() function, into a child of our new main() function. Although it made the final product quite lenghty, having so many abstractions and helper functions ended up being one of our strengths, as it let us more surgically make changes to specific parts of the process, without having to fundementally change the whole flow of the notebook. 